# Скачивание и предобработка текстов

Скачаем тексты сочинений с сайта https://mogu-pisat.ru/sochinenie/.

In [1]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
from datetime import datetime

In [2]:
session = requests.session()

In [3]:
!pip install pyyaml ua-parser user-agents fake-useragent


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
ua = UserAgent(verify_ssl=False)

In [5]:
pages = []

for n in range(2, 50):
    link = 'https://mogu-pisat.ru/sochinenie/ege/?PAGEN_1={}'.format(n)
    headers = {'User-Agent': ua.random}
    response = session.get(link, headers=headers).text
    pages.append(response)
    print(datetime.now())
    time.sleep(1)

2023-10-22 14:33:40.491902
2023-10-22 14:33:41.850710
2023-10-22 14:33:43.169226
2023-10-22 14:33:44.528347
2023-10-22 14:33:45.841383
2023-10-22 14:33:47.209081
2023-10-22 14:33:48.580162
2023-10-22 14:33:49.899040
2023-10-22 14:33:51.265259
2023-10-22 14:33:52.632501
2023-10-22 14:33:53.982938
2023-10-22 14:33:55.311603
2023-10-22 14:33:56.629549
2023-10-22 14:33:57.941310
2023-10-22 14:33:59.260338
2023-10-22 14:34:00.577962
2023-10-22 14:34:01.957523
2023-10-22 14:34:03.323548
2023-10-22 14:34:04.689071
2023-10-22 14:34:06.008837
2023-10-22 14:34:07.401841
2023-10-22 14:34:08.763402
2023-10-22 14:34:10.122616
2023-10-22 14:34:11.503377
2023-10-22 14:34:12.825042
2023-10-22 14:34:14.137947
2023-10-22 14:34:15.516461
2023-10-22 14:34:16.858599
2023-10-22 14:34:18.227130
2023-10-22 14:34:19.614720
2023-10-22 14:34:21.020286
2023-10-22 14:34:22.407643
2023-10-22 14:34:23.729650
2023-10-22 14:34:25.085689
2023-10-22 14:34:26.403696
2023-10-22 14:34:27.764930
2023-10-22 14:34:29.084216
2

In [6]:
len(pages)

48

In [7]:
links = []

for page in pages:
    soup = BeautifulSoup(page, 'html.parser')
    new_links = soup.find_all('div', {'class' : "b-news-item-name"})
    for i in new_links:
        one = i.find('a').get('href')
        links.append(one)

In [13]:
len(links)

960

С помощью краулинга мы собрали ссылки на 960 сочинений. Учитывая, что размер сочинений, как правило, составляет около 200 слов, скачаем 90 сочинений.

In [14]:
essays = dict()
ids = 0

for l in links[:90]:
    full_link = 'https://mogu-pisat.ru' + l
    headers = {'User-Agent': ua.random}
    response = session.get(full_link, headers=headers).text
    soup = BeautifulSoup(response, 'html.parser')
    text = soup.find('div', {"style" : "font-size:17px;"}).get_text()
    title = soup.title.get_text()
    author = soup.find_all('div', {"style" : "font-size:16px;"})[1].get_text()
    data = [text, title, author, full_link]
    essays.update({ids : data})
    ids += 1
    print(datetime.now())
    time.sleep(1)

2023-10-22 14:39:30.625743
2023-10-22 14:39:31.912861
2023-10-22 14:39:33.149899
2023-10-22 14:39:34.405091
2023-10-22 14:39:35.695579
2023-10-22 14:39:36.913652
2023-10-22 14:39:38.469564
2023-10-22 14:39:39.760471
2023-10-22 14:39:41.018930
2023-10-22 14:39:42.266418
2023-10-22 14:39:43.543801
2023-10-22 14:39:44.782256
2023-10-22 14:39:46.079700
2023-10-22 14:39:47.368269
2023-10-22 14:39:48.619841
2023-10-22 14:39:49.862448
2023-10-22 14:39:51.149636
2023-10-22 14:39:52.402537
2023-10-22 14:39:53.680204
2023-10-22 14:39:54.932063
2023-10-22 14:39:56.364497
2023-10-22 14:39:57.893346
2023-10-22 14:39:59.214480
2023-10-22 14:40:00.517411
2023-10-22 14:40:01.753321
2023-10-22 14:40:02.992797
2023-10-22 14:40:04.319028
2023-10-22 14:40:05.559746
2023-10-22 14:40:06.813549
2023-10-22 14:40:08.099768
2023-10-22 14:40:09.391995
2023-10-22 14:40:10.727886
2023-10-22 14:40:12.003827
2023-10-22 14:40:13.255667
2023-10-22 14:40:14.525179
2023-10-22 14:40:15.823203
2023-10-22 14:40:17.112650
2

In [15]:
len(essays)

90

Essays - словарь, в котором ключами являются id сочинений, а значениями - информация о них: 
1. текст сочинения
2. название сочинения
3. автор
4. ссылка на сайт

Сохраним получившийся корпус в файл формата json:

In [16]:
import json
with open('corpus.json', 'w', encoding="utf-8") as fp:
    json.dump(essays, fp, ensure_ascii=False)

Разделим сочинения на предложения:

In [17]:
from nltk import sent_tokenize

def cleaning(text):
    cleaning = "\n\xa0«»\t—…" + "'‘’"
    new_text = ''

    for ch in text:
        if ch not in cleaning:
            new_text += ch
    sent_tokens = sent_tokenize(new_text)
    return sent_tokens

In [18]:
id_sent = 0
sentences = dict()

for essay in essays.items():
    key = essay[0]
    content = essay[1][0]
    sents = cleaning(content)
    for s in sents:
        sent_doc = [s, key]
        sentences.update({id_sent : sent_doc})
        id_sent += 1

In [19]:
len(sentences)

1527

Мы создали словарь, в котором ключами являются id предложений. Значения - текст предложения и id сочинения, из которого оно было извлечено. 

Длина корпуса - 1527 предложений

In [20]:
sentences[0]

[' В предложенном для анализа тексте Владимир Николаевич Крупин размышляет о взаимодействии природы и человека.',
 0]

In [21]:
with open('corpus_sent.json', 'w', encoding="utf-8") as f:
    json.dump(sentences, f, ensure_ascii=False)

# Частеречная разметка корпуса

Для морфологической разметки корпуса было решено использовать библиотеку pymorphy по нескольким причинам:
* относительно высокая точноть разметки и производительность
* удобный тегсет, специализированный для русского языка
* несколько вариантов морфологического разбора с вероятностью каждого -> возможность разрешать или не разрешать морфологическую неоднозначность в особых случаях

In [22]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [23]:
from nltk import word_tokenize

**Разрешение морфологической неоднозначности**

Было принято решение в отдельных случаях в морфологической разметке приписывать одной и той же словоформе нескольких разборов. К этим случаям относятся ситуации, когда определение части речи зависит от контекста, ср.:
* *так* - наречие или частица
* формы прилагательных, совпадающие с формой причастий (*открытый* и под.)

Pymorphy указывает вероятность того, что тот или иной разбор правильный (score). В случаях, когда score >= 0,6, словоформе приписывает один, наиболее вероятный тег. В остальных случаях словоформе приписывается несколько возможных разборов.

In [24]:
def tagging(word):
    parsed = morph.parse(word.lower())
    wordform = parsed[0].word
    lemma = [parsed[0].normal_form]
    pos = [str(parsed[0].tag).split(',')[0].split(' ')[0]]
    if parsed[0].score < 0.6:
        for option in parsed:
            other_tag = str(option.tag).split(',')[0].split(' ')[0]
            if other_tag not in pos:
                pos.append(other_tag)
                lemma.append(option.normal_form)
        return wordform, lemma, pos
    else:
        return wordform, lemma, pos

In [25]:
tokens = dict()
token_id = 0

for sent in sentences.items():
    sent_id = sent[0]
    sentence = sent[1][0]
    words = word_tokenize(sentence)
    for w in words:
        word_info = tagging(w)
        num_tags = len(word_info[2])
        if num_tags > 1:
            for i in range(num_tags):
                info = {'wordform' : word_info[0],
                        'lemma' : word_info[1][i],
                        'pos' : word_info[2][i],
                        'sent_id' : sent_id}
                tokens.update({token_id : info})
                token_id += 1
        else:
            info = {'wordform' : word_info[0],
                    'lemma' : word_info[1][0],
                    'pos' : word_info[2][0],
                    'sent_id' : sent_id}
            tokens.update({token_id : info})
            token_id += 1

**Корпус токенов**

Мы создали словарь, в котором ключами являются token_id. По каждому ключу выдаётся следующая информация о словоформе:

* сама словоформа
* лемма
* pos
* id предложения, в котором она содержится

В случаях, когда словоформа имеет несколько вариантов разбора, каждый разбор хранится под отедльным token_id.

Словоформа с единственным разбором:

In [26]:
tokens[0]

{'wordform': 'в', 'lemma': 'в', 'pos': 'PREP', 'sent_id': 0}

Словоформа с двумя вариантами разбора:

In [27]:
print(tokens[23])
print(tokens[24])

{'wordform': 'следующие', 'lemma': 'следующий', 'pos': 'ADJF', 'sent_id': 1}
{'wordform': 'следующие', 'lemma': 'следовать', 'pos': 'PRTF', 'sent_id': 1}


In [28]:
len(tokens)

32009

Всего корпус содержит 32009 токенов (включая пунктуацию).

In [29]:
with open('corpus_tokens.json', 'w', encoding='utf-8') as file:
    json.dump(tokens, file, ensure_ascii=False)